In [1]:
#!/usr/bin/python3
import requests
import json
import yaml
import ipaddress
import sys
import os
from subprocess import PIPE, Popen
from pathlib import Path
def extract_member(base_json):
    return list(map(lambda x:x["value"],filter(lambda x:x["name"] == "members", base_json["objects"]["object"][0]["attributes"]["attribute"])))
def pack_member(base_json,member_list):
    atlist = base_json["objects"]["object"][0]["attributes"]["attribute"]
    atlist = list(filter(lambda x:x["name"] != "members",atlist))
    atlist = atlist[0:3] + [{"name": "members", "value": member, "referenced-type":"aut-num" if member[:2] == "AS" and member[2:].isdecimal() else "as-set" } for member in member_list] + atlist[3:]
    base_json["objects"]["object"][0]["attributes"]["attribute"] = atlist
    return base_json
def getval(strin):
    return strin.split(":",1)[1].strip()
    
def getAddr(addr):
    addr = addr.strip()
    if "%" in addr:
        addr = addr.split("%",1)
    else:
        addr = addr, None
    return ipaddress.ip_address(addr[0]) , addr[1]
    
def getAddrFromChannel(birdspaline):
    birdspaline = birdspaline.strip()
    if " " in birdspaline:
        addr,ll = birdspaline.split(" ",1)
        if addr == "::":
            return ipaddress.ip_address(ll)
        return ipaddress.ip_address(addr)
    return ipaddress.ip_address(birdspaline)
    
def getroutecount(birdspaline):
    birdspaline = birdspaline.strip()
    infos_list = list( map( lambda x:x.strip(), birdspaline.split(",")))
    infos =  {"imported": None,"filtered":None,"exported": None,"preferred": None}
    for info in infos_list:
        val,key = info.strip().split(" ")
        val = int(val)
        infos[key] = val
    return infos
    
def get_bird_session(n="*",birdc_output = None):
    if n == "*":
        n = '"*"'
    if birdc_output == None:
        birdc_output = Popen(["birdc", "s", "p","a",n], stdin=PIPE, stdout=PIPE).communicate()[0].decode()
    birdc_output = birdc_output.split("\n")[2:]
    birdc_output = "\n".join(birdc_output).split("\n\n")
    result_list = []
    for proto_str in birdc_output:
        proto_str_line = proto_str.split("\n")
        protoinfo = proto_str_line[0].strip().split()
        if len(protoinfo) < 3:
            continue
        proto_name, proto_type, proto_table ,proto_state , proto_since ,proto_info = protoinfo[0] , protoinfo[1], protoinfo[2], protoinfo[3], protoinfo[4], protoinfo[-1]
        if proto_type != "BGP":
            continue
        result = {"name": proto_name, "state":None, "as": {"local":0, "remote":0}, "addr":{"af": 0, "local":None, "remote":None, "interface":None}, "route":{"ipv4":{"imported":0,"exported":0,"preferred":0},"ipv6":{"imported":0,"exported":0,"preferred":0}}}
        current_channel = ""
        for L in proto_str_line:
            if "BGP state:" in L:
                result["state"] = getval(L)
            elif "Neighbor AS:" in L:
                result["as"]["remote"] = int(getval(L))
            elif "Local AS" in L:
                result["as"]["local"] = int(getval(L))
            elif "Neighbor address:" in L:
                remote = getval(L)
                addrobj,interface = getAddr(remote)
                result["addr"]["interface"] = interface
                result["addr"]["remote"] = str(addrobj)
                if type(addrobj) == ipaddress.IPv4Address:
                    result["addr"]["af"] = 4
                elif type(addrobj) == ipaddress.IPv6Address:
                    result["addr"]["af"] = 6
            elif "Channel" in L:
                current_channel = L.split("Channel ")[1].strip()
            elif "Routes:" in L:
                result["route"][current_channel] = getroutecount(getval(L))
            elif "BGP Next hop:" in L:
                if (result["addr"]["af"] == 4 and current_channel == "ipv4") or (result["addr"]["af"] == 6 and current_channel == "ipv6"):
                    result["addr"]["local"] = str(getAddrFromChannel(getval(L)))
        result_list += [result]
    #return yaml.safe_dump(result_list)
    return result_list

In [2]:
def session_to_dict(sess_info):
    result = {}
    for sess in sess_info:
        key = sess["as"]["remote"]
        if key in result:
            result[key] += [sess]
        else:
            result[key] = [sess]
    return result

In [12]:
RS1_info = session_to_dict(get_bird_session())

RS2_birdc = requests.get("http://192.168.16.102:3234/bird?q=show+protocols+all").text
RS2_info = session_to_dict(get_bird_session(birdc_output=RS2_birdc))

RS3_birdc = requests.get("http://192.168.16.103:3234/bird?q=show+protocols+all").text
RS3_info = session_to_dict(get_bird_session(birdc_output=RS3_birdc))

In [13]:
RS2_info[149835]

[{'name': 'AS149835_1',
  'state': 'Passive',
  'as': {'local': 114514, 'remote': 149835},
  'addr': {'af': 6,
   'local': None,
   'remote': 'fe80::149:835',
   'interface': 'eth1'},
  'route': {'ipv4': {'imported': 0, 'exported': 0, 'preferred': 0},
   'ipv6': {'imported': 0, 'exported': 0, 'preferred': 0}}}]

In [21]:
all_conf = yaml.safe_load( open("/root/arouteserver/clients_all.yml").read())

In [40]:
status = []
as_num = {}
def get_dict(default,d,klist):
    for k in klist:
        if k in d:
            d = d[k]
        else:
            return default
    return d
def add_dict(d,k,v):
    if k in d:
        d[k] += [v]
    else:
        d[k] = [v]

for i in range(len(all_conf["clients"])):
    item = {}
    conf = all_conf["clients"][i]
    asn = conf["asn"]
    index = len(get_dict([],as_num,[asn]))
    add_dict(as_num,asn,i)
    item["ASN"] = conf["asn"]
    item["Name"] = f'[{ conf["name"] }](https://bgp.tools/as/{asn})'
    item["AS-SET"] = ", ".join(conf["cfg"]["filtering"]["irrdb"]["as_sets"])
    try:
        rs1 = RS1_info[asn][index]
        item["RS1"] = rs1["route"]["ipv6"]["imported"] if rs1["state"] == "Established" else "-"
    except:
        item["RS1"]  = "x"
    try:
        rs2 = RS2_info[asn][index]
        item["RS2"] = rs2["route"]["ipv6"]["imported"] if rs2["state"] == "Established" else "-"
    except:
        item["RS2"]  = "x"
    try:
        rs3 = RS3_info[asn][index]
        item["RS3"] = rs3["route"]["ipv6"]["imported"] if rs3["state"] == "Established" else "-"
    except:
        item["RS3"]  = "x"
    item["Transit service"] = "✔️" if get_dict("",conf["cfg"]["filtering"],["transit_free","action"]) == "tag" else ""
    status += [item]


In [41]:
def tomark(listOfDicts):
    """Loop through a list of dicts and return a markdown table as a multi-line string.
    listOfDicts -- A list of dictionaries, each dict is a row
    """
    markdowntable = ""
    # Make a string of all the keys in the first dict with pipes before after and between each key
    markdownheader = '| ' + ' | '.join(map(str, listOfDicts[0].keys())) + ' |'
    # Make a header separator line with dashes instead of key names
    markdownheaderseparator = '|-----' * len(listOfDicts[0].keys()) + '|'
    # Add the header row and separator to the table
    markdowntable += markdownheader + '\n'
    markdowntable += markdownheaderseparator + '\n'
    # Loop through the list of dictionaries outputting the rows
    for row in listOfDicts:
        markdownrow = ""
        for key, col in row.items():
            markdownrow += '| ' + str(col) + ' '
        markdowntable += markdownrow + '|' + '\n'
    return markdowntable

In [42]:
print(tomark(status))

| ASN | Name | AS-SET | RS1 | RS2 | RS3 | Transit service |
|-----|-----|-----|-----|-----|-----|-----|
| 138517 | [Poema Network](https://bgp.tools/as/138517) | AS-POEMA | 1 | 1 | x |  |
| 138517 | [Poema Network(KSKB-IX)](https://bgp.tools/as/138517) | AS-POEMA | 1 | - | x |  |
| 142553 | [Air's Project Private Network](https://bgp.tools/as/142553) | AS-AIRSPROJECT | 5 | - | 5 |  |
| 207705 | [JK-Network](https://bgp.tools/as/207705) | AS-JKNET | 4 | 4 | x |  |
| 51930 | [Haonan Yang](https://bgp.tools/as/51930) | AS-AUGUST | 0 | - | 1 |  |
| 135395 | [Hertz Global Network](https://bgp.tools/as/135395) | AS-HERTZ | 1 | - | x |  |
| 210320 | [ZH Innovation Network](https://bgp.tools/as/210320) | AS-ZH-NET | - | - | x |  |
| 213262 | [Moe Overflow Electric](https://bgp.tools/as/213262) | AS-CUTE | - | - | x |  |
| 141011 | [HUIZE-NET](https://bgp.tools/as/141011) | AS-HUIZE | 18 | - | x |  |
| 141237 | [Moecast Network](https://bgp.tools/as/141237) | AS-MOEC | - | - | x |  |
| 139317 |